In [2]:
import pandas as pd
import joblib   
import numpy as np
from sklearn.model_selection import train_test_split
from SimulateDay import scale_data, predict_action, get_stock_data, add_columns, stock_market_simulation
import xgboost as xgb
import joblib
import altair as alt
LGBM_decisions = pd.read_csv('simResults/LGBM_model_decisions.csv')
new_results = pd.read_csv('simResults/sim_results.csv')
specific_model_decisions = pd.read_csv('simResults/specific_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])
general_model_decisions = pd.read_csv('simResults/general_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])

In [66]:
specific_model_decisions.head(10)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
0,A,0,Hold,144.800003,10000.000000,0.0,10000.0,2024-10-09
1,A,1,Sell,142.740005,10000.000000,0.0,10000.0,2024-10-10
2,A,2,Sell,143.820007,10000.000000,0.0,10000.0,2024-10-11
3,A,3,Sell,144.250000,10000.000000,0.0,10000.0,2024-10-14
4,A,4,Sell,144.580002,10000.000000,0.0,10000.0,2024-10-15
2712,A,5,Sell,138.830002,10000.000000,0.0,10000.0,2024-10-16
3210,A,5,Sell,138.245697,10000.000000,0.0,10000.0,2024-10-17
3712,A,7,Sell,138.774994,10000.000000,0.0,10000.0,2024-10-18
5,AAPL,0,Buy,229.539993,9770.460007,1.0,10000.0,2024-10-09
6,AAPL,1,Buy,229.039993,9541.420013,2.0,9999.5,2024-10-10


In [3]:
LGBM_decisions[LGBM_decisions['Action'] == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
4,MMM,4,Buy,135.679993,9864.320007,1,10000.000000,2024-10-17
10,AOS,2,Buy,80.949997,9919.050003,1,10000.000000,2024-10-15
11,AOS,3,Buy,80.230003,9838.820000,2,9999.280006,2024-10-16
20,ABT,4,Buy,117.889999,9882.110001,1,10000.000000,2024-10-17
21,ABT,5,Buy,117.889999,9764.220001,2,10000.000000,2024-10-17
...,...,...,...,...,...,...,...,...
4006,ZBRA,6,Buy,373.839996,8871.100006,3,9992.619995,2024-10-17
4007,ZBRA,7,Buy,375.450012,8495.649994,4,9997.450043,2024-10-18
4014,ZBH,6,Buy,104.949997,9895.050003,1,10000.000000,2024-10-17
4020,ZTS,4,Buy,192.389999,9807.610001,1,10000.000000,2024-10-15


In [6]:
LGBM_decisions.to_csv('simResults/LGBM_model_decisions.csv', index=False)

In [58]:
specific_model_decisions[specific_model_decisions['Action']  == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
5,AAPL,0,Buy,229.539993,9770.460007,1.0,10000.000000,2024-10-09
6,AAPL,1,Buy,229.039993,9541.420013,2.0,9999.500000,2024-10-10
22,ACGL,1,Buy,113.900002,9886.099998,1.0,10000.000000,2024-10-10
26,ACN,0,Buy,365.079987,9634.920013,1.0,10000.000000,2024-10-09
30,ACN,4,Buy,368.660004,9266.260010,2.0,10003.580017,2024-10-15
...,...,...,...,...,...,...,...,...
2465,ZBH,1,Buy,102.820000,9897.180000,1.0,10000.000000,2024-10-10
2468,ZBH,4,Buy,104.599998,9895.400002,1.0,10000.000000,2024-10-15
2829,ZBH,5,Buy,105.209999,9894.790001,1.0,10000.000000,2024-10-16
3327,ZBH,5,Buy,105.029999,9894.970001,1.0,10000.000000,2024-10-17


In [69]:
specific_model_decisions[(specific_model_decisions['Portfolio Value'] > 10000) & (specific_model_decisions['Day'] == 7)]

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
3479,AAPL,7,Sell,235.244995,10005.705002,0.0,10005.705002,2024-10-18
3515,ACN,7,Buy,375.660004,9259.260010,2.0,10010.580017,2024-10-18
3661,AEP,7,Buy,100.625000,9801.654999,2.0,10002.904999,2024-10-18
3907,AKAM,7,Buy,106.485001,9790.404999,2.0,10003.375000,2024-10-18
3688,AMP,7,Buy,522.979980,8983.780029,2.0,10029.739990,2024-10-18
...,...,...,...,...,...,...,...,...
3946,TFX,7,Buy,238.520004,9527.069992,2.0,10004.110001,2024-10-18
3622,TGT,7,Buy,155.550003,9690.119995,2.0,10001.220001,2024-10-18
3510,TMO,7,Hold,601.643982,9400.750000,1.0,10002.393982,2024-10-18
3896,TXT,7,Buy,89.915001,9823.745003,2.0,10003.575005,2024-10-18


In [70]:
specific_model_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,3981.000000,3981.000000,3981.000000,3981.000000,3981.000000
mean,3.513690,222.987359,9892.693099,0.470291,10000.670648
std,2.292576,526.585729,521.677924,0.787780,9.091135
min,0.000000,7.520000,0.000000,0.000000,9857.950195
25%,2.000000,69.570000,9926.240002,0.000000,10000.000000
50%,4.000000,125.839996,10000.000000,0.000000,10000.000000
75%,6.000000,233.020004,10000.000000,1.000000,10000.000000
max,7.000000,9916.605469,10089.529785,4.000000,10289.005859


In [4]:
LGBM_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,4024.000000,4024.000000,4024.000000,4024.000000,4024.000000
mean,3.500000,223.215227,9880.923389,0.593936,9999.695030
std,2.291573,523.721807,359.670803,0.996451,4.691022
min,0.000000,7.520000,226.419922,0.000000,9953.529968
25%,1.750000,69.627499,9893.474998,0.000000,10000.000000
50%,3.500000,126.895000,10000.000000,0.000000,10000.000000
75%,5.250000,233.245003,10000.000000,1.000000,10000.000000
max,7.000000,9924.400391,10000.709999,10.000000,10150.820312


In [9]:
modelComparison = pd.read_csv('simResults/MODELCOMPARISON.csv', index_col=0)
modelComparison.sort_values(by=['Total Profit %'], ascending=False).head(10)


,Model,Stock,Final Portfolio Value,Total Profit %
5,Stacking,AAPL,12560.38,25.60
1,Random Forest,AAPL,12438.13,24.38
3,XGBoost,AAPL,12404.69,24.04
4,Specific Model,AAPL,12404.69,24.04
0,Logistic Regression,AAPL,12010.10,20.01
9,XGBoost,MSFT,11808.35,18.08
8,SVM,MSFT,11803.34,18.03
10,Specific Model,MSFT,11774.20,17.74
6,Logistic Regression,MSFT,11677.37,16.77
7,Random Forest,MSFT,11187.02,11.87


In [10]:
import altair as alt

chart = alt.Chart(modelComparison).mark_bar().encode(
    x=alt.X('Stock:N', title='Tested Stocks'),
    y=alt.Y('Total Profit %:Q', title='Total Profit %'),
    color='Model:N',
    column='Model:O'
).properties(
    width=100,
    height=800,
    title='Total Profit % by Model Over Year To date Simulation',
    center=True
).configure_axis(
    labelAngle=45
)

chart.display()


alt.Chart(...)

In [11]:
def get_final_portfolio_values(df):
    # Group by 'Stock Name' and get the last row for each group
    final_values = df.groupby('Stock Name').apply(lambda x: x.iloc[-1])
    
    # Extract 'Stock Name' and 'Portfolio Value' columns
    result = final_values[['Stock Name', 'Portfolio Value','Shares Held']].reset_index(drop=True)
    
    return result


In [27]:
final_portfolio_values = get_final_portfolio_values(new_results)#.drop(simResults[simResults['Stock Name']=='NFLX'].index))
final_portfolio_values['Profit %'] = (final_portfolio_values['Portfolio Value'] - 10000) / 10000 * 100
alt.Chart(final_portfolio_values).mark_bar().encode(
    x='Stock Name',
    y='Profit %',
    color=alt.condition(
        alt.datum['Profit %'] > 0,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    title='Final Portfolio Value by Stock',
    width=800,
    height=400
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [28]:
final_portfolio_values.describe() , final_portfolio_values0.describe()

(       Portfolio Value  Shares Held    Profit %
 count        21.000000    21.000000   21.000000
 mean      10972.932935    19.012682    9.729329
 std        2596.431139    35.059624   25.964311
 min        9109.066677    -0.980334   -8.909333
 25%        9941.999969     0.000000   -0.580000
 50%       10001.850000     8.000000    0.018500
 75%       10996.178911    17.590241    9.961789
 max       21447.965016   153.364062  114.479650,
        Portfolio Value  Shares Held   Profit %
 count        21.000000    21.000000  21.000000
 mean      10891.237419    18.016460   8.912374
 std        2343.959668    34.993873  23.439597
 min        8714.090503    -4.850322 -12.859095
 25%        9927.830093     0.000000  -0.721699
 50%        9986.659996     5.000000  -0.133400
 75%       10897.731785    18.295126   8.977318
 max       19849.520512   150.832216  98.495205)

In [16]:
import plotly.subplots as sp
import plotly.graph_objects as go

def plot_portfolio_value_and_close_price(df, symbol):
    df = df[df['Stock Name'] == symbol]
    fig = sp.make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Scatter(
            x=df['Day'],
            y=df['Portfolio Value'],
            mode='lines',
            name='Portfolio Value'
        )
    )

    fig.add_trace(go.Scatter(
            x=df['Day'],
            y=df['Stock Price'],
            mode='lines',
            name='Stock Price'
        ), row=2, col=1
    )

    fig.add_trace(go.Bar(
            x=df['Day'],
            y=df['Shares Held'],
            name='Shares Held'
        ), row=2, col=1
    )

    fig.update_layout(
        title=f'Portfolio Value and Close Price for {symbol}',
        xaxis_title='Day',
        yaxis_title='Value',
        hovermode='x unified', # Compare data points on hover

    )

    fig.show()


In [34]:

plot_portfolio_value_and_close_price(new_results, 'GOOGL')

In [5]:
new_results['Profit %'] = (new_results['Portfolio Value'] - 10000) / 10000 * 100
new_results.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value,Profit %
count,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000
mean,3.000000,213.717239,9656.508071,0.857143,9993.913880,-0.060861
std,2.006838,176.402568,1165.791153,1.747797,40.266798,0.402668
min,0.000000,17.250000,2734.600220,0.000000,9669.130371,-3.308696
25%,1.000000,102.705002,9837.070007,0.000000,10000.000000,0.000000
50%,3.000000,174.669998,10000.000000,0.000000,10000.000000,0.000000
75%,5.000000,248.112503,10000.000000,1.000000,10000.000000,0.000000
max,6.000000,765.119995,10000.000000,10.000000,10133.950195,1.339502


In [19]:
final_portfolio_values.describe()

,Portfolio Value,Shares Held,Profit %
count,21.000000,21.000000,21.000000
mean,10972.932935,19.012682,9.729329
std,2596.431139,35.059624,25.964311
min,9109.066677,-0.980334,-8.909333
25%,9941.999969,0.000000,-0.580000
50%,10001.850000,8.000000,0.018500
75%,10996.178911,17.590241,9.961789
max,21447.965016,153.364062,114.479650


In [20]:
simResults

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,Profit %
0,AAPL,0,Buy,167.449997,9162.750015,5.000000,10000.000000,2023-05-03,0.000000
1,AAPL,1,Buy,165.789993,8333.800049,10.000000,9991.699982,2023-05-04,-0.083000
2,AAPL,2,Hold,173.570007,8333.800049,10.000000,10069.500122,2023-05-05,0.695001
3,AAPL,3,Buy,173.500000,8160.300049,11.000000,10068.800049,2023-05-08,0.688000
4,AAPL,4,Buy,171.770004,7988.530045,12.000000,10049.770096,2023-05-09,0.497701
...,...,...,...,...,...,...,...,...,...
7660,UNH,360,Buy,581.580017,9104.739807,0.019666,9116.177131,2024-10-08,-8.838229
7661,UNH,361,Sell,591.219971,9695.959778,-0.980334,9116.366709,2024-10-09,-8.836333
7662,UNH,362,Sell,597.700012,9695.959778,-0.980334,9110.014104,2024-10-10,-8.899859
7663,UNH,363,Buy,598.049988,9097.909790,0.019666,9109.671011,2024-10-11,-8.903290


In [54]:
from SimulateDay import scale_and_obtain_data
x_train, x_test, y_train, y_test = scale_and_obtain_data('TSLA')

Adding columns...
Splitting data...


In [55]:
x_train

,Volume,MA_10,MA_20,MA_50,MA_200,std_10,std_20,std_50,std_200,upper_band_10,...,Support_10_Day,Resistance_20_Day,Support_20_Day,Resistance_50_Day,Support_50_Day,Volume_MA_10,Volume_MA_20,Volume_MA_50,OBV,Z-score
975,0.162461,0.025673,0.028007,0.031513,0.023386,0.027001,0.018512,0.018770,0.048819,0.027281,...,0.025039,0.028752,0.026732,0.031445,0.030102,0.560756,0.534773,0.523430,0.159180,0.022026
517,0.009757,0.004659,0.004891,0.005571,0.006043,0.002641,0.001624,0.003405,0.002402,0.004550,...,0.004213,0.004672,0.004498,0.005682,0.005065,0.054056,0.046805,0.052897,0.016924,0.004353
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011866,0.000000
3578,0.082547,0.447783,0.485109,0.533170,0.765692,0.135730,0.246528,0.274363,0.421332,0.416454,...,0.450588,0.494280,0.481059,0.554236,0.541703,0.196466,0.252300,0.317890,0.905742,0.416689
2844,0.116807,0.630846,0.617386,0.649471,0.607028,0.207425,0.271086,0.347157,0.833888,0.589580,...,0.625758,0.619818,0.603213,0.663561,0.625597,0.219077,0.259873,0.333388,0.731405,0.585157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,0.066370,0.038714,0.040824,0.039623,0.041446,0.007884,0.009683,0.019820,0.031953,0.035323,...,0.040143,0.039037,0.042858,0.039037,0.039689,0.179821,0.228830,0.277625,0.212092,0.035313
1304,0.081638,0.034447,0.036193,0.038401,0.050234,0.009727,0.008770,0.011219,0.021046,0.031910,...,0.035182,0.035305,0.037561,0.036703,0.042296,0.178670,0.240166,0.228654,0.189215,0.031505
870,0.140779,0.017410,0.017668,0.014143,0.009668,0.005401,0.006713,0.028159,0.020205,0.016214,...,0.017463,0.017941,0.017217,0.017941,0.009303,0.265720,0.456189,0.458807,0.119932,0.017022
3517,0.102015,0.648476,0.655237,0.649824,0.747099,0.186245,0.128323,0.295375,0.489358,0.601278,...,0.657193,0.627412,0.697402,0.627412,0.657911,0.253364,0.311206,0.371161,0.946658,0.615996
